In [1]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# prompt: import excel file

import pandas as pd

# Replace 'your_excel_file.xlsx' with the actual path to your Excel file in Google Drive
excel_file_path = '/content/drive/Shareddrives/Capstone/preprocessing_wip_Maite/zero_shot_transformers.xlsx'

df = pd.read_excel(excel_file_path)

df.head(2)

,classification_result
0,"{""sequence"": ""My name is Walter Hartwell White..."
1,"{""sequence"": ""White Residence"", ""labels"": [""po..."


In [3]:
import pandas as pd
import json
import re

# Function to fix single quotes and parse JSON, while preserving single quotes inside the text
def fix_and_parse_json(x):
    try:
        # Use a regex pattern that avoids replacing single quotes inside words like "you're" or "won't"
        # We target only the single quotes around property names and values.
        # This pattern ensures that quotes around keys and values are replaced, but not within the sequence text.
        fixed_str = re.sub(r"(?<!\w)'(?!\w)", '"', x)

        # Parse the corrected JSON string
        return json.loads(fixed_str)
    except (json.JSONDecodeError, TypeError) as e:
        print(f"Error parsing JSON: {e}\nError at entry: {x}")
        return None  # Handle if parsing fails or if it's not a string

# Apply the function to the 'classification_result' column
df['classification_result'] = df['classification_result'].apply(lambda x: fix_and_parse_json(x) if isinstance(x, str) else x)

# Extract the 'sequence' value from the parsed 'classification_result' column
df['sequence'] = df['classification_result'].apply(lambda x: x['sequence'] if isinstance(x, dict) else None)

# Print the first few rows to verify
print(df[['classification_result', 'sequence']].head())

Error parsing JSON: Expecting ',' delimiter: line 1 column 36 (char 35)
Error at entry: {"sequence": "Well, too bad, man, "cause guess what. He's still not dead! I went and put my ear to the RV, I can hear him, like, rolling around in there, all right? I think he's awake, man, I think he's trying to get loose. Where the hell are you? I'm freaking out over here!", "labels": ["negative", "neutral", "positive"], "scores": [0.9449244737625122, 0.03199154883623123, 0.023084037005901337]}
Error parsing JSON: Expecting ',' delimiter: line 1 column 29 (char 28)
Error at entry: {"sequence": "So, the term "chiral"derives from the Greek word "hand. "The concept here being that, just as your left hand, and your right hand are mirror images of one another, right? Identical, and yet opposite. Well, so too organic compounds can exist as mirror image forms of one another all the way down at the molecular level. But although they may look the same, they don\'t always behave the same. For instance...I\'

In [4]:
# Now extract 'labels', 'scores', and 'sequence' from the dictionary
df['labels'] = df['classification_result'].apply(lambda x: x['labels'] if isinstance(x, dict) else None)
df['scores'] = df['classification_result'].apply(lambda x: x['scores'] if isinstance(x, dict) else None)
df['sequence'] = df['classification_result'].apply(lambda x: x['sequence'] if isinstance(x, dict) else None)

# Display the result
print(df[['sequence', 'labels', 'scores']].head())

                                            sequence  \
0  My name is Walter Hartwell White. I live at 30...   
1                                    White Residence   
2                              (Three weeks earlier)   
3                                    Happy Birthday.   
4                                      Look at that.   

                          labels  \
0  [neutral, negative, positive]   
1  [positive, neutral, negative]   
2  [negative, neutral, positive]   
3  [positive, neutral, negative]   
4  [positive, neutral, negative]   

                                              scores  
0  [0.6377227306365967, 0.22701147198677063, 0.13...  
1  [0.597480833530426, 0.32724621891975403, 0.075...  
2  [0.40771177411079407, 0.3248295485973358, 0.26...  
3  [0.9430945515632629, 0.03763766214251518, 0.01...  
4  [0.7173102498054504, 0.1923591047525406, 0.090...  


In [5]:
df = df.drop(columns=['classification_result','sequence'])
df.head(2)

,labels,scores
0,"[neutral, negative, positive]","[0.6377227306365967, 0.22701147198677063, 0.13..."
1,"[positive, neutral, negative]","[0.597480833530426, 0.32724621891975403, 0.075..."


In [6]:
df['label_score_pairs'] = df.apply(
    lambda row: list(zip(row['labels'], row['scores'])) if row['labels'] is not None and row['scores'] is not None else None,
    axis=1
)

# Display the result with label-score pairs
print(df['label_score_pairs'].head())

0    [(neutral, 0.6377227306365967), (negative, 0.2...
1    [(positive, 0.597480833530426), (neutral, 0.32...
2    [(negative, 0.40771177411079407), (neutral, 0....
3    [(positive, 0.9430945515632629), (neutral, 0.0...
4    [(positive, 0.7173102498054504), (neutral, 0.1...
Name: label_score_pairs, dtype: object


In [7]:
# Create columns for positive, negative, and neutral scores
df['positive'] = 0.0
df['negative'] = 0.0
df['neutral'] = 0.0

# Assign scores to the respective columns based on labels
for index, row in df.iterrows():
    if row['label_score_pairs'] is not None:
        for label, score in row['label_score_pairs']:
            if label == 'positive':
                df.at[index, 'positive'] = score
            elif label == 'negative':
                df.at[index, 'negative'] = score
            elif label == 'neutral':
                df.at[index, 'neutral'] = score

# Display the result with the new columns
print(df[['positive', 'negative', 'neutral']].head())

   positive  negative   neutral
0  0.135266  0.227011  0.637723
1  0.597481  0.075273  0.327246
2  0.267459  0.407712  0.324830
3  0.943095  0.019268  0.037638
4  0.717310  0.090331  0.192359
